<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20neuronal%20con%20text%20embeddings%2C%20features%20e%20hyperparameter%20automatic%20tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras.models
import keras.layers as layers
from keras.models import Sequential

from google.colab import files

!pip install -q tensorflow-hub
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)

!pip install talos
import talos
import datetime as dt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Version:  1.14.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [2]:
# CARGAMOS LOS CSV
url_train_con_features = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv'
url_test_con_features = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv'

train_df = pd.read_csv(url_train_con_features)
test_df = pd.read_csv(url_test_con_features)

train_df.head()

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [3]:
# LIMPIAMOS Y CHUSMEAMOS
train_df = train_df.drop(['Unnamed: 0', 'keyword', 'location'], axis = 1)
train_df.shape

(7613, 17)

In [4]:
target_df = train_df.pop('target')

In [5]:
# HACEMOS UN EMBEDDING PROVISTO POR TensorFlow Hub, por fuera del modelo, de esa forma puedo agregarle features al array

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_array = np.asarray(train_df['text'])
train_text_embedded = hub_layer(train_text_array)
train_text_embedded_array = train_text_embedded.numpy()
train_text_embedded_df = pd.DataFrame(data = train_text_embedded_array)

train_text_embedded_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.007239,0.156896,1.295307,-0.040780,-0.160369,-1.200443,-0.404513,-0.523973,-0.235587,-0.350222,-0.289882,-0.471979,-0.076219,0.170489,-0.832377,0.022601,0.161255,-0.640868,-0.569074,0.040001
1,-0.498373,1.282888,-0.188361,-0.947347,0.880581,-0.178855,-0.015432,0.383270,-1.508848,1.753281,-0.754933,0.601241,-0.143784,-0.891944,0.432137,1.346741,-1.485925,1.576216,-0.378781,0.942368
2,0.088144,-0.311402,0.604415,1.713081,1.320312,-3.201303,0.844513,0.962179,-1.535314,2.206953,-0.279025,0.010960,-2.228273,0.204312,-2.175010,1.344161,0.118840,-1.238801,-0.255011,0.695272
3,-1.083202,0.694350,-0.293344,0.301429,1.887291,-2.864778,-0.018864,0.176138,-2.388204,1.020881,0.144147,0.802330,-1.326284,-0.055402,-0.268148,1.400147,-1.432996,-1.546905,0.456249,0.117063
4,0.296060,-0.401119,0.260654,-0.133400,0.043096,-0.781329,-0.456910,0.952776,0.359138,0.342683,-1.734295,1.611835,-0.623508,-0.758112,-0.305311,1.009357,-0.759674,-0.749319,-1.216560,0.413135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.851313,1.155238,-0.563759,0.338048,1.038724,-0.160978,-0.569604,0.691393,-1.027794,-0.184893,-0.310249,1.777828,-1.333608,-1.097076,-0.556300,1.235541,-0.814311,-0.244320,-0.842489,0.383638
7609,0.586044,1.064535,1.108355,-1.080643,0.978821,-1.538974,-0.011408,0.165366,-0.385796,-0.280098,0.166554,-0.024055,-0.658078,-0.365682,-0.477021,1.355311,-0.433643,0.137504,-1.611735,-0.096220
7610,-1.030287,0.200154,-0.691702,-0.506228,0.250602,0.100836,-0.644162,-1.211905,-1.112232,0.593809,-0.752241,0.449338,0.918436,-0.354921,1.325370,-0.197728,-1.613481,0.592320,1.134160,0.184020
7611,0.192334,-0.517235,-0.762114,-0.045759,2.537380,-0.443210,-0.915873,1.407808,0.293244,-0.688244,-2.480736,1.473341,0.463894,-0.913488,-1.088880,2.593801,-0.944662,-0.532709,-1.173615,0.009434


In [6]:
# MANDALE MECHA CON LOS FEATURES MUÑECO
train_df = train_df.drop(['id', 'text'],axis = 1)
train_df = pd.concat([train_text_embedded_df, train_df], axis=1)

train_df['has_mentions'] = train_df['has_mentions'].astype(int)
train_df['has_link'] = train_df['has_link'].astype(int)
train_df['contains_countries'] = train_df['contains_countries'].astype(int)
train_df['contains_cities'] = train_df['contains_cities'].astype(int)

train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0.007239,0.156896,1.295307,-0.040780,-0.160369,-1.200443,-0.404513,-0.523973,-0.235587,-0.350222,-0.289882,-0.471979,-0.076219,0.170489,-0.832377,0.022601,0.161255,-0.640868,-0.569074,0.040001,69,13,0,0,0,0,0,0,0,0,0,7,0,0
1,-0.498373,1.282888,-0.188361,-0.947347,0.880581,-0.178855,-0.015432,0.383270,-1.508848,1.753281,-0.754933,0.601241,-0.143784,-0.891944,0.432137,1.346741,-1.485925,1.576216,-0.378781,0.942368,38,7,1,0,0,0,0,0,0,0,0,3,1,0
2,0.088144,-0.311402,0.604415,1.713081,1.320312,-3.201303,0.844513,0.962179,-1.535314,2.206953,-0.279025,0.010960,-2.228273,0.204312,-2.175010,1.344161,0.118840,-1.238801,-0.255011,0.695272,133,22,0,0,0,0,0,0,0,0,0,13,0,0
3,-1.083202,0.694350,-0.293344,0.301429,1.887291,-2.864778,-0.018864,0.176138,-2.388204,1.020881,0.144147,0.802330,-1.326284,-0.055402,-0.268148,1.400147,-1.432996,-1.546905,0.456249,0.117063,65,8,1,0,0,0,0,0,0,0,0,2,0,0
4,0.296060,-0.401119,0.260654,-0.133400,0.043096,-0.781329,-0.456910,0.952776,0.359138,0.342683,-1.734295,1.611835,-0.623508,-0.758112,-0.305311,1.009357,-0.759674,-0.749319,-1.216560,0.413135,88,16,1,0,0,0,0,0,0,0,0,10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.851313,1.155238,-0.563759,0.338048,1.038724,-0.160978,-0.569604,0.691393,-1.027794,-0.184893,-0.310249,1.777828,-1.333608,-1.097076,-0.556300,1.235541,-0.814311,-0.244320,-0.842489,0.383638,83,11,0,0,0,0,0,0,0,0,1,6,0,0
7609,0.586044,1.064535,1.108355,-1.080643,0.978821,-1.538974,-0.011408,0.165366,-0.385796,-0.280098,0.166554,-0.024055,-0.658078,-0.365682,-0.477021,1.355311,-0.433643,0.137504,-1.611735,-0.096220,125,20,1,0,0,0,0,1,0,0,0,13,0,0
7610,-1.030287,0.200154,-0.691702,-0.506228,0.250602,0.100836,-0.644162,-1.211905,-1.112232,0.593809,-0.752241,0.449338,0.918436,-0.354921,1.325370,-0.197728,-1.613481,0.592320,1.134160,0.184020,65,8,0,0,0,0,0,0,0,1,1,4,0,0
7611,0.192334,-0.517235,-0.762114,-0.045759,2.537380,-0.443210,-0.915873,1.407808,0.293244,-0.688244,-2.480736,1.473341,0.463894,-0.913488,-1.088880,2.593801,-0.944662,-0.532709,-1.173615,0.009434,137,19,0,0,0,0,0,0,0,0,0,12,1,1


In [7]:
# SE DIVIDE UNA PARTE DEL SET PARA PODER EVALUAR LOS RESULTADOS

#train_x, validation_x, train_y, validation_y = train_test_split(train_df, target_df, test_size=0.3, random_state=0)

In [8]:
def modelardo(x_train, y_train, x_test, y_test, params):
  
  input_dim = x_train.shape[1]
  model = Sequential()

  model.add(layers.Dense(params['first neuron units'], 
                         input_dim = input_dim,
                         activation = params['first neuron activation']))
  
  model.add(layers.Dense(1, 
                         activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy', 
                optimizer = 'adam', 
                metrics = ['accuracy'])
  
  out = model.fit(x_train, 
                  y_train,
                  epochs = params['epochs'],
                  verbose = 0, 
                  validation_data = (x_test, y_test), 
                  batch_size = params['batch size'])

  return out, model


p = {
    'first neuron units': [1, 2, 20],
    'first neuron activation': ['relu', 'elu', 'sigmoid', 'softmax', 'tanh', 'exponential'],
    'epochs': [1, 2, 3, 4, 5, 6, 7, 10, 20, 100],
    'batch size': [5, 50, 100]
}

In [9]:
# ESTO FALLA
t = talos.Scan(train_df.to_numpy(),
               target_df.to_numpy(),
               params = p,
               model = modelardo,
               experiment_name = 'diabetes',
               time_limit = (dt.datetime.now() + dt.timedelta(minutes = 10)).strftime("%Y-%m-%d %H:%M"),
               random_method = 'quantum')

  0%|          | 0/540 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


RuntimeError: ignored

In [ ]:
# UNA VEZ SOLUCIONADO LO DE ARRIBA, DE ACA PARA ABAJO HAY QUE VER QUE PASA
experiment_data = t.data
experiment_data.head()

In [ ]:
# TODO: ACA HAY QUE PONER LOS HIPERPARAMETROS QUE SE ENCONTRARON EN LA BUSQUEDA

# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_validation, y_train, y_validation = train_test_split(data_train,train_df['target'],random_state=7,test_size = 0.20)

input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Dense(2,input_dim=input_dim,activation='elu'))
best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(x_train,y_train,epochs = 5,verbose = 0,validation_data = (x_validation,y_validation),batch_size=50)

loss, accuracy = best_model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = best_model.evaluate(x_validation, y_validation, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
# EVALUAMOS CON EL SET DE VALIDACION
loss, precision = model.evaluate(validation_x, validation_y)
print('Precisión: %.2f' % (precision*100))

In [ ]:
# TODO: ACA HAY QUE PONER LOS HIPERPARAMETROS QUE SE ENCONTRARON EN LA BUSQUEDA
# SE REENTRENA CON EL SET COMPLETO

out = best_model.fit(data_train,
                     train_df['target'],
                     epochs = 5,
                     verbose = 1,
                     batch_size = 62)
data_test = vectorizer.transform(test_df['text']).toarray()


In [ ]:
# ARMAMOS LA PREDICCIÓN
prediction = best_model.predict_classes(data_test, batch_size = 512)
prediction

In [ ]:
# DOWNLOAD !
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')